In [ ]:
%matplotlib qt
import mne
import os
from mne.preprocessing import ICA
import sys
print('Python v{}'.format(sys.version))
print('MNE v{}'.format(mne.__version__))

In [ ]:
epochs_fname=('epochs_fif/')#define raw file name
subjectlist=os.listdir(epochs_fname)#get each subject name
#os.makedirs('ica_fif/')

In [ ]:
NB_list,NG_list,NN_list=list(),list(),list()
YB_list,YG_list,YN_list=list(),list(),list()

In [ ]:
for subject in subjectlist:#loop with subject
    epochs_name_path=os.path.join(epochs_fname,subject)#Concatenate file name 
    epochs=mne.read_epochs(epochs_name_path,preload=True)#read epochs
    ica = ICA(n_components=20, max_pca_components=20, 
              method='fastica',random_state=98).fit(epochs)#run ica base on first 20 PCA components 
    ica.plot_sources(epochs,block=True)#plot 20 ICA components and remove artifacts EOG and ECG
    ica.apply(epochs)#apply ICA to epochs
    epochs.apply_baseline(None,0)#baseline correction
    No_Bad,No_Good,No_Normal=epochs['No_Bad'],epochs['No_Good'],epochs['No_Normal']
    Yes_Bad,Yes_Good,Yes_Normal=epochs['Yes_Bad'],epochs['Yes_Good'],epochs['Yes_Normal']#create epochs for each condition
    NB_list.append(No_Bad.average())#create evoked list for each condition
    NG_list.append(No_Good.average())
    NN_list.append(No_Normal.average())
    YB_list.append(Yes_Bad.average())
    YG_list.append(Yes_Good.average())
    YN_list.append(Yes_Normal.average())

In [ ]:
picks = [epochs.ch_names.index(ch) for ch in ["PO4","PO3","PO7","PO8"]]#define electrode of interest
condition_index={'No_Bad':NB_list,'No_Good':NG_list,'No_Normal':NN_list,
                'Yes_Bad':YB_list,'Yes_Good':YG_list,'Yes_Normal':YN_list} #set parameter for plot ERP

In [ ]:
mne.viz.plot_compare_evokeds(condition_index,picks=picks,ci=None,legend='lower right',show_sensors='upper right')#plot compared ERP  